In [8]:
import numpy as np
import sympy as sym
%matplotlib widget
import matplotlib.pyplot as plt

In [9]:
v_tm,delta_t,phi_t = sym.symbols('v_tm,delta_t,phi_t', real=True)
v_gm,delta_g,phi_g = sym.symbols('v_gm,delta_g,phi_g', real=True)
v_tr,v_ti,v_sr,v_si,v_gr,v_gi = sym.symbols('v_tr,v_ti,v_sr,v_si,v_gr,v_gi', real=True)
R_s,X_s,R_g,X_g =  sym.symbols('R_s,X_s,R_g,X_g', real=True)
i_sr,i_si = sym.symbols('i_sr,i_si', real=True)

In [10]:
j = sym.I

v_t = v_tm*sym.exp(j*(delta_t + phi_t))
v_g = v_gm*sym.exp(j*(delta_g + phi_g))

print(f'v_tr = {str(sym.simplify(sym.re(v_t)))}')
print(f'v_ti = {str(sym.simplify(sym.im(v_t)))}')
print(f'v_gr = {str(sym.simplify(sym.re(v_g)))}')
print(f'v_gi = {str(sym.simplify(sym.im(v_g)))}')

v_t = v_tr + j*v_ti
v_s = v_sr + j*v_si
v_g = v_gr + j*v_gi
Z_s = R_s + j*X_s
Z_g = R_g + j*X_g

i_s = (v_t - v_g)/(Z_s + Z_g)
print(f'i_sr = {str(sym.simplify(sym.re(i_s)))}')
print(f'i_si = {str(sym.simplify(sym.im(i_s)))}')

i_s  = i_sr + j*i_si
v_s = v_t - i_s*Z_s
print(f'v_sr = {str(sym.simplify(sym.re(v_s)))}')
print(f'v_si = {str(sym.simplify(sym.im(v_s)))}')

v_s = v_sr + j*v_si
s_s =v_s*sym.conjugate(i_s)
print(f'p_s = {str(sym.simplify(sym.re(s_s)))}')
print(f'q_s = {str(sym.simplify(sym.im(s_s)))}')

v_tr = v_tm*cos(delta_t + phi_t)
v_ti = v_tm*sin(delta_t + phi_t)
v_gr = v_gm*cos(delta_g + phi_g)
v_gi = v_gm*sin(delta_g + phi_g)
i_sr = (-(R_g + R_s)*(v_gr - v_tr) - (X_g + X_s)*(v_gi - v_ti))/((R_g + R_s)**2 + (X_g + X_s)**2)
i_si = (-(R_g + R_s)*(v_gi - v_ti) + (X_g + X_s)*(v_gr - v_tr))/((R_g + R_s)**2 + (X_g + X_s)**2)
v_sr = -R_s*i_sr + X_s*i_si + v_tr
v_si = -R_s*i_si - X_s*i_sr + v_ti
p_s = i_si*v_si + i_sr*v_sr
q_s = -i_si*v_sr + i_sr*v_si


In [4]:
ddelta_t = Omega_b*(omega_t - omega_coi)
ddelta_g = Omega_b*(omega_g - omega_coi)
dDomega_g = RoCoF

NameError: name 'Omega_b' is not defined

In [ ]:
Dt = 0.01
times = np.arange(0,10,Dt)

# parameters
R_s = 0.01
X_s = 0.1
R_g = 0.001
X_g = 0.01
Omega_b = 2*np.pi*50

# initial inputs
omega_t = 1.0
RoCoF = 0.0
v_tm = 1.0
phi_t = 0.0
v_gm = 1.0
phi_g = 0.0


# initial states
delta_t = 0.0
delta_g = 0.0
Domega_g = 0.0

xi_p_s = 0.0
xi_q_s = 0.0

# record
P_s = []
Q_s = []
Phi_t = []

for t in times:
        
    # plant
    omega_g = 1.0 + Domega_g
    omega_coi = omega_g

    delta_t  = delta_t  + Dt*Omega_b*(omega_t - omega_g)
    delta_g  = delta_g  + Dt*Omega_b*(omega_g - omega_coi)
    Domega_g = Domega_g + Dt*RoCoF

    v_tr = v_tm*np.cos(delta_t + phi_t)
    v_ti = v_tm*np.sin(delta_t + phi_t)
    v_gr = v_gm*np.cos(delta_g + phi_g)
    v_gi = v_gm*np.sin(delta_g + phi_g)
    i_sr = -((R_g + R_s)*(v_gr - v_tr) + (X_g + X_s)*(v_gi - v_ti))/((R_g + R_s)**2 + (X_g + X_s)**2)
    i_si = (-(R_g + R_s)*(v_gi - v_ti) + (X_g + X_s)*(v_gr - v_tr))/((R_g + R_s)**2 + (X_g + X_s)**2)
    v_sr = -R_s*i_sr + X_s*i_si + v_tr
    v_si = -R_s*i_si - X_s*i_sr + v_ti
    p_s  =  i_si*v_si + i_sr*v_sr
    q_s  = -i_si*v_sr + i_sr*v_si
    v_sm = (v_gr**2+v_gi**2)**0.5
    
    # control
    p_s_ref = 0.0
    q_s_ref = 0.0
    v_gm = 1.0
    RoCoF = 0.0
    if t>1.0:
        p_s_ref = 0.8
    if t>3.0:
        q_s_ref = 0.4       
    if t>5.0:
        v_gm = 0.95    
    if t>7.0:
        RoCoF = -0.001    
    if t>9.0:
        RoCoF =  0.0 
        
    epsilon_p_s = (p_s_ref - p_s)
    phi_t = 0.01*epsilon_p_s + 1.0*xi_p_s 
    xi_p_s = xi_p_s + Dt*epsilon_p_s
    omega_t = omega_g
    
    epsilon_q_s = (q_s_ref - q_s)
    v_tm = 0.01*epsilon_q_s + 1.0*xi_q_s + 1.0
    xi_q_s = xi_q_s + Dt*epsilon_q_s
    
    print(f't = {t:0.2f}, p_s = {p_s:0.2f}, q_s = {q_s:0.2f}')
    
    P_s += [p_s]
    Q_s += [q_s]
    Phi_t += [phi_t]
        

In [129]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 8))


axes[0].plot(times,P_s)
axes[0].plot(times,Q_s)
axes[1].plot(times,Phi_t)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [231]:
Dt = 0.02
times = np.arange(0,10,Dt)

# parameters
R_s = 0.01
X_s = 0.1
R_g = 0.001
X_g = 0.01
Omega_b = 2*np.pi*50

# initial inputs
omega_t = 1.0
RoCoF = 0.0
v_tm = 1.0
phi_t = 0.0
v_gm = 1.0
phi_g = 0.0
theta_t = 0.0

# initial states
delta_t = 0.0
delta_g = 0.0
Domega_g = 0.0

xi_p_s = 0.0
xi_q_s = 0.0
xi_p_s2 = 0.0

# record
P_s = []
Q_s = []
Phi_t = []
Theta_t  = []
Omega_t = []
V_sd,V_sq = [],[]
I_sd,I_sq = [],[]

for t in times:
        
    # plant
    omega_g = 1.0 + Domega_g
    omega_coi = omega_g

    delta_t  = delta_t  + Dt*Omega_b*(omega_t - omega_g)
    #delta_g  = delta_g  + Dt*Omega_b*(omega_g - omega_g)
    Domega_g = Domega_g + Dt*RoCoF

    v_tr = v_tm*np.cos(delta_t + phi_t)
    v_ti = v_tm*np.sin(delta_t + phi_t)
    v_gr = v_gm*np.cos(delta_g + phi_g)
    v_gi = v_gm*np.sin(delta_g + phi_g)
    i_sr = -((R_g + R_s)*(v_gr - v_tr) + (X_g + X_s)*(v_gi - v_ti))/((R_g + R_s)**2 + (X_g + X_s)**2)
    i_si = (-(R_g + R_s)*(v_gi - v_ti) + (X_g + X_s)*(v_gr - v_tr))/((R_g + R_s)**2 + (X_g + X_s)**2)
    v_sr = -R_s*i_sr + X_s*i_si + v_tr
    v_si = -R_s*i_si - X_s*i_sr + v_ti
    p_s  =  i_si*v_si + i_sr*v_sr
    q_s  = -i_si*v_sr + i_sr*v_si
    v_sm = (v_gr**2+v_gi**2)**0.5
    # r -> -Q, i -> D
    v_sd = v_si*np.cos(delta_t) + v_sr*np.sin(delta_t)
    v_sq = v_si*np.sin(delta_t) - v_sr*np.cos(delta_t)
    i_sd = i_si*np.cos(delta_t) + i_sr*np.sin(delta_t)
    i_sq = i_si*np.sin(delta_t) - i_sr*np.cos(delta_t)
    
    
    # control
    p_s_ref = 0.0
    q_s_ref = 0.0
    v_gm = 1.0
    RoCoF = 0.0
    if t>1.0:
        p_s_ref = 0.8
    if t>3.0:
        q_s_ref = 0.4       
    if t>5.0:
        v_gm = 0.95    
    if t>7.0:
        RoCoF = -0.001    
    if t>9.0:
        RoCoF =  0.0 
        
    epsilon_p_s = (p_s_ref - p_s)
    omega_t = 0.01*epsilon_p_s + 0.05*xi_p_s + 1.0
    #phi_t   = 0.05*epsilon_p_s + 2.0*xi_p_s + 5.0*xi_p_s2 
    xi_p_s = xi_p_s + Dt*epsilon_p_s
    xi_p_s2 = xi_p_s2 + Dt*xi_p_s

    
    epsilon_q_s = (q_s_ref - q_s)
    v_tm = 0.01*epsilon_q_s + 1.0*xi_q_s + 1.0
    xi_q_s = xi_q_s + Dt*epsilon_q_s
    
    print(f't = {t:0.2f}, p_s = {p_s:0.2f}, q_s = {q_s:0.2f}')
    
    P_s += [p_s]
    Q_s += [q_s]
    Phi_t += [phi_t]
    Theta_t += [theta_t]   
    Omega_t += [omega_t]
    V_sd += [v_sd]
    V_sq += [v_sq]
    I_sd += [i_sd]
    I_sq += [i_sq]

t = 0.00, p_s = 0.00, q_s = -0.00
t = 0.02, p_s = 0.00, q_s = -0.00
t = 0.04, p_s = 0.00, q_s = -0.00
t = 0.06, p_s = 0.00, q_s = -0.00
t = 0.08, p_s = 0.00, q_s = -0.00
t = 0.10, p_s = 0.00, q_s = -0.00
t = 0.12, p_s = 0.00, q_s = -0.00
t = 0.14, p_s = 0.00, q_s = -0.00
t = 0.16, p_s = 0.00, q_s = -0.00
t = 0.18, p_s = 0.00, q_s = -0.00
t = 0.20, p_s = 0.00, q_s = -0.00
t = 0.22, p_s = 0.00, q_s = -0.00
t = 0.24, p_s = 0.00, q_s = -0.00
t = 0.26, p_s = 0.00, q_s = -0.00
t = 0.28, p_s = 0.00, q_s = -0.00
t = 0.30, p_s = 0.00, q_s = -0.00
t = 0.32, p_s = 0.00, q_s = -0.00
t = 0.34, p_s = 0.00, q_s = -0.00
t = 0.36, p_s = 0.00, q_s = -0.00
t = 0.38, p_s = 0.00, q_s = -0.00
t = 0.40, p_s = 0.00, q_s = -0.00
t = 0.42, p_s = 0.00, q_s = -0.00
t = 0.44, p_s = 0.00, q_s = -0.00
t = 0.46, p_s = 0.00, q_s = -0.00
t = 0.48, p_s = 0.00, q_s = -0.00
t = 0.50, p_s = 0.00, q_s = -0.00
t = 0.52, p_s = 0.00, q_s = -0.00
t = 0.54, p_s = 0.00, q_s = -0.00
t = 0.56, p_s = 0.00, q_s = -0.00
t = 0.58, p_s 

t = 7.60, p_s = 0.82, q_s = 0.40
t = 7.62, p_s = 0.82, q_s = 0.40
t = 7.64, p_s = 0.82, q_s = 0.40
t = 7.66, p_s = 0.82, q_s = 0.40
t = 7.68, p_s = 0.82, q_s = 0.40
t = 7.70, p_s = 0.82, q_s = 0.40
t = 7.72, p_s = 0.82, q_s = 0.40
t = 7.74, p_s = 0.82, q_s = 0.40
t = 7.76, p_s = 0.82, q_s = 0.40
t = 7.78, p_s = 0.82, q_s = 0.40
t = 7.80, p_s = 0.82, q_s = 0.40
t = 7.82, p_s = 0.82, q_s = 0.40
t = 7.84, p_s = 0.82, q_s = 0.40
t = 7.86, p_s = 0.82, q_s = 0.40
t = 7.88, p_s = 0.82, q_s = 0.40
t = 7.90, p_s = 0.82, q_s = 0.40
t = 7.92, p_s = 0.82, q_s = 0.40
t = 7.94, p_s = 0.82, q_s = 0.40
t = 7.96, p_s = 0.82, q_s = 0.40
t = 7.98, p_s = 0.82, q_s = 0.40
t = 8.00, p_s = 0.82, q_s = 0.40
t = 8.02, p_s = 0.82, q_s = 0.40
t = 8.04, p_s = 0.82, q_s = 0.40
t = 8.06, p_s = 0.82, q_s = 0.40
t = 8.08, p_s = 0.82, q_s = 0.40
t = 8.10, p_s = 0.82, q_s = 0.40
t = 8.12, p_s = 0.82, q_s = 0.40
t = 8.14, p_s = 0.82, q_s = 0.40
t = 8.16, p_s = 0.82, q_s = 0.40
t = 8.18, p_s = 0.82, q_s = 0.40
t = 8.20, 

In [233]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 8))
axes[0].plot(times,V_sd)
axes[0].plot(times,V_sq)
axes[1].plot(times,I_sd)
axes[1].plot(times,I_sq)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 8))
axes[0].plot(times,P_s)
axes[0].plot(times,Q_s)
axes[1].step(times,Omega_t)

In [199]:
plt.close('all')

In [222]:
400e6/len(times)

1600000.0

In [ ]:
1_600_000